In [64]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [65]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1041,tt1142977,39000000,81491068,Frankenweenie,Charlie Tahan|Winona Ryder|Catherine O'Hara|Ma...,Tim Burton,The electrifying dog is back from beyond the g...,Young Victor conducts a science experiment to ...,87,Animation|Comedy|Family,Walt Disney Pictures|Tim Burton Animation Comp...,10/4/2012,6.5,2012
1749,tt0462244,6000000,18197398,Daddy Day Camp,Cuba Gooding Jr.|Tamala Jones|Paul Rae|Lochlyn...,Fred Savage,The summer is going to be in tents.,Seeking to offer his son the satisfying summer...,89,Comedy|Family,Revolution Studios|TriStar Pictures,8/8/2007,4.8,2007
1798,tt0499554,10000000,20342161,Reno 911!: Miami,Lennie Loftin|Danny DeVito|Ben Garant|Carlos A...,Ben Garant,"In a time of crisis, America will call Reno 911!",A rag-tag team of Reno cops are called in to s...,84,Action|Adventure|Comedy|Crime,20th Century Fox,2/23/2007,5.8,2007
126,tt1790864,34000000,348319861,The Maze Runner,Dylan O'Brien|Ki Hong Lee|Kaya Scodelario|Aml ...,Wes Ball,Run - Remember - Survive,"Set in a post-apocalyptic world, young Thomas ...",113,Action|Mystery|Science Fiction|Thriller,Ingenious Media|Twentieth Century Fox Film Cor...,9/10/2014,7.0,2014
429,tt1477076,20000000,136150434,Saw 3D,Tobin Bell|Costas Mandylor|Betsy Russell|Cary ...,Kevin Greutert,The Traps Come Alive,As a deadly battle rages over Jigsaw's brutal ...,90,Horror|Crime,Lionsgate|Twisted Pictures|Serendipity Product...,10/21/2010,5.8,2010


In [66]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [154]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

data['profit'] = np.subtract(data['revenue'], data['budget'])
import re
import operator
def genres_max (gen):
    genres = []
    flat_list = []
    for genre in gen:
        genres.append(genre.split("|"))

    for sublist in genres:
        for item in sublist:
            flat_list.append(item)
    genres_count = dict(Counter(flat_list))
    res = max(genres_count.items(), key=operator.itemgetter(1))[0]

    return res

def genres_profit (gen):
    genres = []
    flat_list = []
    for genre in gen['genres'][gen['profit']>0]:
        genres.append(genre.split("|"))

    for sublist in genres:
        for item in sublist:
            flat_list.append(item)
    genres_count = dict(Counter(flat_list))
    res = max(genres_count.items(), key=operator.itemgetter(1))[0]

    return res

def director_max_revenue (gen):
    max_revenue = 0
    director_max = ''
    rev_sum = 0
    for director in gen['director']:
        for rev in gen['revenue'][gen['director']==director]:
            rev_sum+=rev
        if rev_sum>max_revenue:
            max_revenue = rev_sum
            director_max = director
        rev_sum = 0


    return director_max

def director_max_action2 (data):
    dirs = []
    data_action = data[data['genres'].str.contains('Action')]
    for director in data_action['director'].str.split(pat="|"):
        dirs.append(director)

    flat_list = [item for sublist in dirs for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]

    return res

def acter_max_revenue (data):
    acters = []
    acter_max = ""
    count_rev = 0
    count_max = 0
    data_2012 = data[data['release_year'] == 2012]

    for acter in data_2012['cast'].str.split(pat="|"):
        acters.append(acter)

    flat_list = [item for sublist in acters for item in sublist]
    for acter in flat_list:
        for rev in data_2012['revenue'][data_2012['cast'].str.contains(acter)]:
            count_rev+=rev
        if count_rev > count_max:
            count_max = count_rev
            acter_max = acter
        count_rev = 0

    return acter_max

def acter_high (data):
    acters = []
    data_high = data[data['budget'] > data['budget'].mean()]

    for acter in data_high['cast'].str.split(pat="|"):
        acters.append(acter)

    flat_list = [item for sublist in acters for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]

    return res

def acter_cage (data):
    genras = []
    data_cage = data[data['cast'].str.contains("Nicolas Cage")]

    for genre in data_cage['genres'].str.split(pat="|"):
        genras.append(genre)

    flat_list = [item for sublist in genras for item in sublist]
    dirs_count = dict(Counter(flat_list))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]
    return res

def paramount (data):
    genras = []
    profit = 0
    profit_min = 0
    data_paramount = data[data['production_companies'].str.contains("Paramount Pictures")]

    for profit in data_paramount['profit']:
        if profit < profit_min:
            profit_min = profit
    res = data_paramount['original_title'][data_paramount['profit'] == profit_min]


    return res

def year_profit (data):
    years = []
    profits = {}

    for year in data['release_year']:
        years.append(year) if year not in years else years

    for year in years:
        for rev in data['revenue'][data['release_year'] == year]:

            if year in profits:
                profits[year] += rev

            else:
                # profits[year] = int(data['revenue'][data['release_year'] == year])
                profits[year] = rev

    dirs_count = dict(Counter(profits))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def year_profit1 (data):
    years = []
    profits = {}

    for year in data['release_year']:
        years.append(year) if year not in years else years

    for year in years:
        for rev in data['profit'][data['release_year'] == year]:

            if year in profits:
                profits[year] += rev

            else:
                # profits[year] = int(data['revenue'][data['release_year'] == year])
                profits[year] = rev

    dirs_count = dict(Counter(profits))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def month_sum (data):
    data_month = []
    for date in data['release_date']:
        data_month.append(re.search(r'\d+', date).group())

    dirs_count = dict(Counter(data_month))
    sorted_dirs = sorted(dirs_count.items(), key=operator.itemgetter(1))
    res = max(dirs_count.items(), key=operator.itemgetter(1))[0]



    return res

def summer (data):
    count = 0
    for date in data['release_date']:
        if int(re.search(r'\d+', date).group()) == 6 or\
                int(re.search(r'\d+', date).group())== 7 or\
                int(re.search(r'\d+', date).group())==8:
            count+=1


    return count

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [155]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '1. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [156]:
# тут пишем ваш код для решения данного вопроса:
print(data['original_title'][data['budget']==data['budget'].max()])

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


ВАРИАНТ 2

In [157]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [158]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [159]:
print(data['original_title'][data['runtime']==data['runtime'].max()])

1157    Gods and Generals
Name: original_title, dtype: object


# 3. Какой из фильмов самый короткий (в минутах)?





In [160]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [161]:
print(data['original_title'][data['runtime']==data['runtime'].min()])

768    Winnie the Pooh
Name: original_title, dtype: object


# 4. Какова средняя длительность фильмов?


In [162]:
answers['4'] = '4. 110'

In [163]:
print(data['runtime'].mean())

109.6585494970884


# 5. Каково медианное значение длительности фильмов? 

In [164]:
answers['5'] = '5. 107'

In [165]:
print(data['runtime'].median()) #5

107.0


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [166]:
answers['6'] = '6. Avatar (tt0499549)'

In [167]:
# лучше код получения столбца profit вынести в Предобработку что в начале
print(data['original_title'][data['profit']==data['profit'].max()]) #6

239    Avatar
Name: original_title, dtype: object


# 7. Какой фильм самый убыточный? 

In [168]:
answers['7'] = '7. The Lone Ranger (tt1210819)'

In [169]:
print(data['original_title'][data['profit']==data['profit'].min()]) #7

1245    The Lone Ranger
Name: original_title, dtype: object


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [170]:
answers['8'] = '8. 1478'

In [171]:
print(len(data[data['profit']>0]))

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [172]:
answers['9'] = '9. The Dark Knight (tt0468569)'

In [173]:
data2008 = data[data['release_year'] == 2008]
print(data2008['original_title'][data2008['profit'] == data2008['profit'].max()]) #9

599    The Dark Knight
Name: original_title, dtype: object


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [174]:
answers['10'] = '10. The Lone Ranger (tt1210819)'

In [175]:
data2012 = data[(data['release_year']>= 2012) & (data['release_year']<= 2014)]
print(data2012['original_title'][data2012['profit']==data2012['profit'].min()]) #10

1245    The Lone Ranger
Name: original_title, dtype: object


# 11. Какого жанра фильмов больше всего?

In [176]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
answers['11'] = '11. Drama'

In [177]:
print(genres_max(data['genres']))

Drama


ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [178]:
answers['12'] = '12. Drama'

In [179]:
print(genres_profit(data)) #12

Drama


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [180]:
answers['13'] = '13. Peter Jackson'

In [181]:
print(director_max_revenue(data)) #13

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [182]:
answers['14'] = '14. Robert Rodriguez'

In [183]:
print(director_max_action2(data)) #14

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [184]:
answers['15'] = '15. Chris Hemsworth'

In [185]:
print(acter_max_revenue(data)) #15

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [186]:
answers['16'] = '16. Matt Damon'

In [187]:
print(acter_high(data)) #16

Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [188]:
answers['17'] = '17. Action'

In [189]:
print(acter_cage(data)) #17

Action


# 18. Самый убыточный фильм от Paramount Pictures

In [190]:
answers['18'] = '18. K-19: The Widowmaker (tt0267626)'

In [191]:
print(paramount(data)) #18

925    K-19: The Widowmaker
Name: original_title, dtype: object


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [192]:
answers['19'] = '19. 2015'

In [193]:
print(year_profit(data)) #19

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [194]:
answers['20'] = '20. 2014'

In [195]:
data_wb = data[data['production_companies'].str.contains("Warner Bros")]
print(year_profit1(data_wb)) #20

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [196]:
answers['21'] = '21. Сентябрь'

In [197]:
print(month_sum(data)) #21

9


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [199]:
answers['22'] = '22. 450'

In [200]:
print(summer(data)) #22

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [0]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '...', '2': '...'}

In [0]:
# и убедиться что ни чего не пропустил)
len(answers)